In [7]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

ACCOUNT_BUNDLER_LIST = dataiku.Dataset("ACCOUNT_BUNDLER_LIST")
ACCOUNT_BUNDLER_LIST_df = ACCOUNT_BUNDLER_LIST.get_dataframe()
print(len(ACCOUNT_BUNDLER_LIST_df))

NAFCUSTOMER_RDW_CONVERSIONS = dataiku.Dataset("NAFCUSTOMER_RDW_CONVERSIONS")
NAFCUSTOMER_RDW_CONVERSIONS_df = NAFCUSTOMER_RDW_CONVERSIONS.get_dataframe()
print(len(NAFCUSTOMER_RDW_CONVERSIONS_df))

# Read recipe inputs
NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER = dataiku.Dataset("NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER")
NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df = NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER.get_dataframe()
print(len(NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df))



274
2170800
4806903


In [36]:
NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,EDW_CUSTOMER_NAME,ACCOUNT_SINCE_DATE,CUSTOMER_SOURCE_SYSTEM_CODE,ACCOUNT_OPEN_DATE,ACCOUNT_CLOSED_DATE,ATTRITION_TYPE_NAME,ATTRITION_REASON_CODE,ATTRITION_REASON_DESC,YEAR_NUMBER,QUARTER_NUMBER,ACTIVE_CARD_COUNT
0,9100004251546,CAROLINA AIR CARE,WEX FLEET UNIVERSAL PRIMARY,2017-04-27 00:00:00.000,SIEBEL,2017-04-27 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2017,4,15
1,417008246662,U S D 315 COLBY PUBLIC SCHOOLS,U S D 315 COLBY PUBLIC SCHOOLS,2010-05-14 00:00:00.000,TANDEM,2010-05-14 00:00:00.000,2022-09-10 00:00:00.000,NNCL Conversion,04,Account Conversion,2018,2,14
2,430004184602,EASTER SEALS NEW HAMPSHIRE,EASTER SEALS NEW HAMPSHIRE,2000-08-02 00:00:00.000,TANDEM,2000-08-02 00:00:00.000,2022-09-17 00:00:00.000,NNCL Conversion,04,Account Conversion,2019,4,121
3,9100004755591,11300 GRANT COUNTY WA,AMR HOLDCO INC,2018-02-14 00:00:00.000,SIEBEL,2018-02-14 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2019,4,11
4,444007634488,MARTIN CHRYSLER LLC,MARTIN CHRYSLER LLC,2007-12-04 00:00:00.000,TANDEM,2007-12-04 00:00:00.000,2022-04-19 00:00:00.000,NNCL Conversion,04,Account Conversion,2019,2,9


In [41]:
df_a = NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df
df_a['CUSTOMER_ACCOUNT_ID'] = df_a['CUSTOMER_ACCOUNT_ID'].astype('Int64', errors='ignore')
df_a['CUSTOMER_ACCOUNT_NAME'] = df_a['CUSTOMER_ACCOUNT_NAME'].str.upper()
df_a['EDW_CUSTOMER_NAME'] = df_a['EDW_CUSTOMER_NAME'].str.upper()

ACCOUNT_BUNDLER_LIST_df['IS_BUNDLER'] = True
ACCOUNT_BUNDLER_LIST_df = ACCOUNT_BUNDLER_LIST_df[['EDW_CUSTOMER_NAME','IS_BUNDLER']]
ACCOUNT_BUNDLER_LIST_df.head()

df = pd.merge(df_a,ACCOUNT_BUNDLER_LIST_df, how='left', on='EDW_CUSTOMER_NAME')
df.loc[df["IS_BUNDLER"].isnull(),'IS_BUNDLER'] = False
df.CUSTOMER_NAME
df.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,EDW_CUSTOMER_NAME,ACCOUNT_SINCE_DATE,CUSTOMER_SOURCE_SYSTEM_CODE,ACCOUNT_OPEN_DATE,ACCOUNT_CLOSED_DATE,ATTRITION_TYPE_NAME,ATTRITION_REASON_CODE,ATTRITION_REASON_DESC,YEAR_NUMBER,QUARTER_NUMBER,ACTIVE_CARD_COUNT,IS_BUNDLER
0,9100004251546,CAROLINA AIR CARE,WEX FLEET UNIVERSAL PRIMARY,2017-04-27 00:00:00.000,SIEBEL,2017-04-27 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2017,4,15,True
1,417008246662,U S D 315 COLBY PUBLIC SCHOOLS,U S D 315 COLBY PUBLIC SCHOOLS,2010-05-14 00:00:00.000,TANDEM,2010-05-14 00:00:00.000,2022-09-10 00:00:00.000,NNCL Conversion,04,Account Conversion,2018,2,14,False
2,430004184602,EASTER SEALS NEW HAMPSHIRE,EASTER SEALS NEW HAMPSHIRE,2000-08-02 00:00:00.000,TANDEM,2000-08-02 00:00:00.000,2022-09-17 00:00:00.000,NNCL Conversion,04,Account Conversion,2019,4,121,False
3,9100004755591,11300 GRANT COUNTY WA,AMR HOLDCO INC,2018-02-14 00:00:00.000,SIEBEL,2018-02-14 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2019,4,11,False
4,444007634488,MARTIN CHRYSLER LLC,MARTIN CHRYSLER LLC,2007-12-04 00:00:00.000,TANDEM,2007-12-04 00:00:00.000,2022-04-19 00:00:00.000,NNCL Conversion,04,Account Conversion,2019,2,9,False


In [50]:
df['CUSTOMER'] = np.nan
df['CUST_CALC_SOURCE'] = 'Unknown'
df.loc[df.IS_BUNDLER,'EDW_CUSTOMER_NAME'] = np.nan

df.loc[~df['EDW_CUSTOMER_NAME'].isnull(),'CUSTOMER'] = df["EDW_CUSTOMER_NAME"]
df.loc[~df['EDW_CUSTOMER_NAME'].isnull(),'CUST_CALC_SOURCE'] = 'EDW'

df.loc[df['CUSTOMER'].isnull(),'CUST_CALC_SOURCE'] = 'ACCOUNT'
df.loc[df['CUSTOMER'].isnull(),'CUSTOMER'] = df.CUSTOMER_ACCOUNT_NAME

In [51]:
df.CUST_CALC_SOURCE.value_counts(dropna=False)

EDW        2707344
ACCOUNT    2099559
Name: CUST_CALC_SOURCE, dtype: int64

In [42]:
df_g = df.groupby(['EDW_CUSTOMER_NAME']).CUSTOMER_ACCOUNT_ID.nunique().reset_index()
df_g.columns = ['EDW_CUSTOMER_NAME','ACCOUNT_COUNT']
df_g = df_g.sort_values(by=['ACCOUNT_COUNT'], ascending=False)
df_g.head(40)

,EDW_CUSTOMER_NAME,ACCOUNT_COUNT
210826,SHELL SMALL BUSINESS PRIMARY,56872
52024,CHEVRON AND TEXACO BUSINESS PRIMARY,24820
210819,SHELL FLEET PLUS PRIMARY,24417
249938,WEX FLEET UNIVERSAL PRIMARY,17548
52020,CHEVRON TEXACO BUSINESS ACCESS PRIMA,14198
217281,SPEEDWAY BUSINESS FLEET PRIMARY,13486
189452,QUIKTRIP FLEETMASTER PRIMARY,10656
149983,MARATHON FLEET PRIMARY,9009
249937,WEX FLEET FLEXCARD REBATE PRIMARY,8488
247930,WAWA FLEET PRIMARY,7859


In [8]:
ACCOUNT_BUNDLER_LIST_df.head()

,EDW_CUSTOMER_NAME,Distinct count of CUSTOMER_ACCOUNT_ID
0,AUTOMOTIVE RESOURCES INTL HYBRID,6
1,EXXONMOBIL PL CONVERSION L1,1631
2,WEX FLEET UNIVERSAL PRIMARY,77923
3,SHELL SMALL BUSINESS PRIMARY,70865
4,SHELL FLEET PLUS PRIMARY,44536


In [11]:
NAFCUSTOMER_RDW_CONVERSIONS_df.head()
NAFCUSTOMER_RDW_CONVERSIONS_df.STATUS_DATE.min()

'1901-01-01 00:00:00.000'

In [10]:
NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,CUSTOMER_NAME,ACCOUNT_SINCE_DATE,CUSTOMER_SOURCE_SYSTEM_CODE,ACCOUNT_OPEN_DATE,ACCOUNT_CLOSED_DATE,ATTRITION_TYPE_NAME,ATTRITION_REASON_CODE,ATTRITION_REASON_DESC,YEAR_NUMBER,QUARTER_NUMBER,ACTIVE_CARD_COUNT
0,9.100004e+12,Carolina Air Care,WEX Fleet Universal Primary,2017-04-27 00:00:00.000,SIEBEL,2017-04-27 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2017,4,15
1,4.170082e+11,U S D 315 COLBY PUBLIC SCHOOLS,U S D 315 COLBY PUBLIC SCHOOLS,2010-05-14 00:00:00.000,TANDEM,2010-05-14 00:00:00.000,2022-09-10 00:00:00.000,NNCL Conversion,04,Account Conversion,2018,2,14
2,4.300042e+11,EASTER SEALS NEW HAMPSHIRE,EASTER SEALS NEW HAMPSHIRE,2000-08-02 00:00:00.000,TANDEM,2000-08-02 00:00:00.000,2022-09-17 00:00:00.000,NNCL Conversion,04,Account Conversion,2019,4,121
3,9.100005e+12,11300 Grant County WA,AMR HoldCo Inc,2018-02-14 00:00:00.000,SIEBEL,2018-02-14 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2019,4,11
4,4.440076e+11,MARTIN CHRYSLER LLC,MARTIN CHRYSLER LLC,2007-12-04 00:00:00.000,TANDEM,2007-12-04 00:00:00.000,2022-04-19 00:00:00.000,NNCL Conversion,04,Account Conversion,2019,2,9


In [0]:
# Write recipe outputs
#NAFCUSTOMER_LONG_HISTORY_MATCHES = dataiku.Dataset("NAFCUSTOMER_LONG_HISTORY_MATCHES")
#NAFCUSTOMER_LONG_HISTORY_MATCHES.write_with_schema(NAFCUSTOMER_LONG_HISTORY_MATCHES_df)